![monitor](monitor_pic.jpg)


My current computer monitor is getting a bit dated and my eyes have started to feel sore from looking at it all day. I want to buy a new computer monitor, but every time I get online to start searching I get overwhelmed by the number of monitors available. They come in a wide variety of sizes, widths, definitions, display types, port types, etc, and vary in price from under \\$100 to well over \\$1000. While I know that I want to upgrade my screen size to give me more working area, other options, like gaming features, are irrelevant to me. How on earth will I ever be able to decide which monitor is right for me?

This seems like a perfect opportunity to put some data analysis into action! For this project I am going to be acquiring feature data for a number of different computer monitors and running some product analysis across those features to help me make a data driven decision which monitor to purchase. 

### First Step: Get the data
Unfortunately, I was unable to find a downloadable preexisting dataset for computer monitors so I had to make the dataset myself. I looked through a few different websites that carried a wide variety of monitors from different manufacturers but the available data was limited to pretty basic specifications. For this project I wanted to use a dataset with a rich feature set in order to try to explore more subtle differences in features and try to tease out their value. LG is one of the brands that seem to be pretty appealing to me, and when I looked at their website I found that for each monitor they provided over 50 data points and it was in a format that wouldnt be too difficult to scrape. So, my first step of the first step is going to be building a set of functions to scrape, clean and format the data for one monitor into a dataframe. Then later I will use those functions across multiple monitors and compile them all into one dataset. 

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Monitor URL list
I am starting by extracting the URL for each monitor into a list

In [2]:
# extracting the list of monitor URLs to a list
results = requests.get('https://www.lg.com/us/monitors')
content = results.content
soup = BeautifulSoup(content, 'lxml')
monitors = soup.find_all('a', class_='ga-model-detail')

monitor_list = [monitor.get('href') for monitor in monitors]
edited_monitor_list = monitor_list[2:46]
reedited_list = []
for m in edited_monitor_list:
    if m not in reedited_list:
        reedited_list.append(m)
monitor_url_list = [monitor.rsplit('/',1)[1] for monitor in reedited_list]
base_url = 'https://www.lg.com/us/monitors/'
monitor_url_list = [base_url + monitor_extention for monitor_extention in monitor_url_list]

print(f'This is the format for our monitor URLs in the list: {monitor_url_list[0]}')



This is the format for our monitor URLs in the list: https://www.lg.com/us/monitors/lg-27up650-w-4k-uhd-monitor


### Individual monitor data extraction
Next step is building the functions that will extract and clean up the data for each monitor

In [3]:
# downloading the selected spec tags for a given monitor from the LG website

results = requests.get('https://www.lg.com/us/monitors/lg-32ul500-w-4k-uhd-monitor')
content = results.content
parser = BeautifulSoup(content, 'lxml')

table = list(parser.find_all('div', class_="tech-spacs"))

# the result is a list with 10 elements
# each element is a string with spec data for one aspect of the monitor

table_type = type(table)
print(f"returned object is a {table_type} with {len(table)} elements ")

returned object is a <class 'list'> with 10 elements 


In [4]:
# cleaning the first element and turning it into a list

first_table = table[0].text.replace('\n', ' ')
first_table = first_table.replace('\t', ' ')
first_table = first_table.strip()
first_table_listed = first_table.rsplit('   ')

In [5]:
# cleaning up the list

q_list = [element for element in first_table_listed if element != '']
q_header = q_list[0]
q_body = q_list[1:]
f = lambda q: q.strip()
mapped_q = list(map(f, q_body))

In [6]:
#separating the list into category & value lists and recombining as a dictionary

category = [mapped_q[i] for i in range(len(mapped_q)) if i%2 == 0]
value = [mapped_q[i] for i in range(len(mapped_q)) if i%2 != 0]
monitor_dict = dict(zip(category, value))

In [7]:
# putting the values into a dataframe with the row representing the monitor and columns are its attributes

df = pd.DataFrame(data=monitor_dict, index=[0])
df

,Size,Display Type,Response Time,Refresh Rate,Display Resolution,Color Gamut (Typ.),Color Depth (Number of Colors),Pixel Pitch (mm),Aspect Ratio,Resolution,Brightness,Contrast Ratio,Viewing Angle,Surface Treatment
0,"32""",VA,4ms (GtG at Faster),60Hz,4K UHD,DCI-P3 95% (CIE1976),1.07B,0.181x 0.181 mm,16:9,3840 x 2160,300cd/m²,3000:1,"178˚(R/L), 178˚(U/D)",Anti-Glare


In [8]:
# putting it all together and building functions for the transformation of each returned scraped elements

def download_monitor_specs(URL):
    """Import monitor specs and parse into list of strings"""
    
    from bs4 import BeautifulSoup
    import requests
    results = requests.get(URL)
    content = results.content
    parser = BeautifulSoup(content, 'lxml')
    list_of_strings = list(parser.find_all('div', class_="tech-spacs"))
    return list_of_strings
    
def clean_element(list_of_strings):
    """Reformat list of HTML strings and return a list of lists"""
    
    listed_element_lists = []
    for string in list_of_strings:
        string = string.text.replace('\n', ' ')
        string = string.replace('\t', ' ')
        string = string.strip()
        listed_string = string.rsplit('   ')
        listed_element_lists.append(listed_string)
    return listed_element_lists

def clean_listed_element(listed_element_lists):
    """Remove blank lines and strip empty spaces on list elements"""
    
    cleaned_element_list = []
    for list_ in listed_element_lists:
        listed_element = [element for element in list_ if element != '']
        listed_element = listed_element[1:]
        f = lambda q: q.strip()
        listed_element = list(map(f, listed_element))
        cleaned_element_list.append(listed_element)
    return cleaned_element_list

def list_to_dict(cleaned_element_list):
    """Separate list into categories and values then combine into dictionary"""
    
    categories = []
    values = []
    for element in cleaned_element_list:
        category = [element[i] for i in range(len(element)) if i%2 == 0]
        categories += category
        value = [element[i] for i in range(len(element)) if i%2 != 0]
        values+=value
    
    monitor_dict = dict(zip(categories, values))
    return monitor_dict


def dict_to_df(monitor_dict):
    """Convert dictionary of monitor specs into pandas DataFrame object"""
    import pandas as pd
    df = pd.DataFrame(data=monitor_dict, index=[0])
    return df



In [9]:
# testing out the function pipeline so far

url = 'https://www.lg.com/us/monitors/lg-32ul500-w-4k-uhd-monitor'

df = dict_to_df(
    list_to_dict(
    clean_listed_element(
    clean_element(
    download_monitor_specs(url)
    ))))

df.shape

(1, 52)

### Putting together secondary path for review page info retrieval 

In [10]:
# for price, model name and reviews it was easier to scrape from a different URL than the initial scrape

def access_review_page(url):
    """redirect monitor url to monitor review url"""
    monitor_review_url = url + '/reviews'
    
    return monitor_review_url

def extract_review_elements(monitor_review_url):
    """Pulls values from monitor review website and returns dict with 'col':'value' format"""
    results = requests.get(monitor_review_url)
    content = results.content
    soup = BeautifulSoup(content, 'lxml')
    
    values_to_extract = {
        'model_name':'mpn',
        'model_title':'name',
        'price':'price',
        'rating_count':"reviewCount",
        'avg_rating':"ratingValue"
    }
    
    for column in values_to_extract.keys():
        label = values_to_extract[column]
        extracted_html_value = soup.find('meta', {'itemprop':label})
        extracted_value = extracted_html_value.get('content')
        # update dictionary value with extracted value
        values_to_extract[column]=extracted_value
        
    return values_to_extract
        


# Putting it all together
This part uses the list of monitors to retrieve all the data for each monitor and compiles it all together in one pandas dataframe so that I can get to work cleaning the data, and later perform some (hopefully) insightful analysis on it. 

In [11]:
# Initializing a new dataframe outside the for loop to concatonate with
df = pd.DataFrame()

# looping through the URL for each monitor
for monitor in monitor_url_list:
    # building dictionary from values on the monitor page
    d = list_to_dict(
    clean_listed_element(
    clean_element(
    download_monitor_specs(monitor)
    )))
    # building dictionary from values on the monitor review page
    d2 = access_review_page(monitor)
    d2 = extract_review_elements(d2)
    # combining the dictionaries
    d2.update(d)
    # converting the dictionaries into a dataframe
    df2 = pd.DataFrame(data=[d2])
    # adding the monitor's dataframe row to the initial dataframe 
    df = pd.concat([df, df2])
    

In [16]:
# setting up an index
df = df.reset_index(drop=True)
df.shape

(19, 174)

# On to data manipulation!
Finally, I have a working dataframe, now its time to clean it up! Unfortunately there are only 19 monitors for my dataset, which doesnt give me a lot of comparison points to work with. However there are nearly 200 features! An awful lot of those will end up getting removed for lack of utility, and some of them will end up getting combined or transformed, but there should still be plenty of features to work with.